## Example Use of David's Finance Module, dfinance
#### See it in action!
This notebook shows an example that implements my custom finance package to backtest an (ineffective) algorithmic trading strategy.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

import dfinance as dfin
from dfinance.trading_strategies import SMA_Crossover

%load_ext autoreload
%autoreload 2

In [2]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [3]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [4]:
# # load data from Alpha Vantage csv file
# historical_df = pd.read_csv('gme_minute_data.csv', index_col='time')
# historical_df.index = pd.to_datetime(historical_df.index)
# historical_df.index.name = 'datetime'
# historical_df = historical_df.sort_values(by=['datetime'])

# load data from Yahoo!
data = yf.download(tickers='AAPL', period='90d', progress=False)
data.drop(columns=['Adj Close'])
data.to_csv('aapl_daily_data.csv')
historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [5]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                SMA_Crossover.strategy,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='AAPL')

2021-10-04 | Short: 142.39 | Long: 143.43 | Short < Long
2021-10-05 | Short: 142.05 | Long: 142.92 | Short < Long
2021-10-06 | Short: 141.51 | Long: 142.39 | Short < Long
2021-10-07 | Short: 141.22 | Long: 142.09 | Short < Long
2021-10-08 | Short: 140.92 | Long: 141.88 | Short < Long
2021-10-11 | Short: 140.72 | Long: 141.91 | Short < Long
2021-10-12 | Short: 140.65 | Long: 141.80 | Short < Long
2021-10-13 | Short: 140.37 | Long: 141.50 | Short < Long
2021-10-14 | Short: 140.39 | Long: 141.22 | Short < Long
2021-10-15 | Short: 140.83 | Long: 141.09 | Short < Long
2021-10-18 | Short: 141.32 | Long: 141.04 | Short > Long

Buying 10 shares of AAPL for 1431.6000366210938

2021-10-19 | Short: 142.04 | Long: 141.37 | Short > Long
2021-10-20 | Short: 143.02 | Long: 141.77 | Short > Long
2021-10-21 | Short: 143.53 | Long: 142.21 | Short > Long
2021-10-22 | Short: 144.14 | Long: 142.85 | Short > Long
2021-10-25 | Short: 144.72 | Long: 143.47 | Short > Long
2021-10-26 | Short: 145.52 | Long: 144

In [6]:
my_port.cashvalue

100329.3000793457

In [7]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,143.160004,1431.600037,buy,2021-10-18
1,AAPL,-10,178.529999,-1785.299988,sell,2021-12-28
2,AAPL,10,177.259995,1772.599945,buy,2021-12-31
3,AAPL,-10,174.820007,-1748.200073,sell,2022-01-12


In [8]:
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares


In [9]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 3, 100, '2021-01-01 00:00:00')

In [10]:
my_port.save_transactions('my_port.csv')

In [11]:
my_port.load_transactions('my_port.csv')
my_port.summary_of_transactions()

,Ticker,Shares Held,Current Price,Current Value of Shares
1,GME,3,102.669998,308.009995
2,Totals,3,N/A,308.009995
